In [1]:
!pip install dotenv
import os
import time
import pandas as pd
import requests
from typing import List, Dict, Any

import dotenv
dotenv.load_dotenv()
import pandas as pd
import requests
import dotenv
from typing import List, Dict, Any


SEX_DICT = {
    0: "не указан",
    1: "женский",
    2: "мужской"
}

RELATION_DICT = {
    0: "не указано",
    1: "не женат/не замужем",
    2: "есть друг/есть подруга",
    3: "помолвлен/помолвлена",
    4: "женат/замужем",
    5: "всё сложно",
    6: "в активном поиске",
    7: "влюблён/влюблена",
    8: "в гражданском браке"
}

PERSONAL_POLITICAL_DICT = {
    1: "коммунистические",
    2: "социалистические",
    3: "умеренные",
    4: "либеральные",
    5: "консервативные",
    6: "монархические",
    7: "ультраконсервативные",
    8: "индифферентные",
    9: "либертарианские"
}

LIFE_MAIN_DICT = {
    1: "семья и дети",
    2: "карьера и деньги",
    3: "развлечения и отдых",
    4: "наука и исследования",
    5: "совершенствование мира",
    6: "саморазвитие",
    7: "красота и искусство",
    8: "слава и влияние"
}

SMOKING_DICT = {
    1: "резко негативное",
    2: "негативное",
    3: "компромиссное",
    4: "нейтральное",
    5: "положительное"
}

ALCOHOL_DICT = SMOKING_DICT.copy()

def load_access_token(token_arg) -> str:

    token = os.getenv("VK_SERVICE_TOKEN", token_arg)
    if not token:
        raise ValueError("Не удалось загрузить токен. Проверьте файл .env и переменную VK_SERVICE_TOKEN или передайте токен как переменную.")
    return token


def get_members(params: Dict[str, Any]) -> List[Dict[str, Any]]:

    members = []
    offset = 0

    while True:
        params['offset'] = offset


        response = requests.get('https://api.vk.com/method/groups.getMembers', params=params).json()

        if 'error' in response:
            print(f"Ошибка: {response['error']['error_msg']}")
            break

        items = response['response']['items']
        members.extend(items)


        if len(items) < params['count']:
            break

        offset += params['count']
        time.sleep(0.5)

    return members


def fetch_groups_members(group_ids: List[str], access_token: str) -> Dict[str, List[Dict[str, Any]]]:

    parsing_result = {"members": []}
    for group_id in group_ids:
        params = {
            'access_token': access_token,
            'v': '5.131',
            'group_id': group_id,
            'fields': 'id,first_name,last_name,sex,bdate,city,country,relation,personal,connections,education,'
                      'universities,occupation,career,military,interests',
            'count': 1000
        }
        group_members = get_members(params)


        parsing_result["members"].extend(group_members)

    return parsing_result


def flatten_member(member: Dict[str, Any]) -> Dict[str, Any]:


    universities = member.get("universities", [])
    if len(universities) > 0:
        university_name = universities[0].get("name")
        faculty = universities[0].get("faculty_name")
        graduation = universities[0].get("graduation")
    else:
        university_name = None
        faculty = None
        graduation = None


    career = member.get("career", [])
    if len(career) > 0:
        company = career[0].get("company")
        position = career[0].get("position")
    else:
        company = None
        position = None


    military_list = member.get("military", [])
    if len(military_list) > 0:
        military = military_list[0].get("unit")
    else:
        military = None

    return {
        "id": member.get("id"),
        "first_name": member.get("first_name"),
        "last_name": member.get("last_name"),
        "sex": SEX_DICT.get(member.get("sex")),
        "bdate": member.get("bdate"),
        "city": member.get("city", {}).get("title"),
        "country": member.get("country", {}).get("title"),
        "relation": RELATION_DICT.get(member.get("relation")),
        "interests": member.get("personal", {}).get("interests"),
        "political": PERSONAL_POLITICAL_DICT.get(member.get("personal", {}).get("political")),
        "life_main": LIFE_MAIN_DICT.get(member.get("personal", {}).get("life_main")),
        "smoking": SMOKING_DICT.get(member.get("personal", {}).get("smoking")),
        "alcohol": ALCOHOL_DICT.get(member.get("personal", {}).get("alcohol")),
        "connections": member.get("connections"),
        "university_name": university_name,
        "faculty": faculty,
        "graduation": graduation,
        "company": company,
        "position": position,
        "military": military,
        "occupation": member.get("occupation", {}).get("name"),
    }


def flatten_members_data(members_data: Dict[str, Any]) -> pd.DataFrame:

    members_list = members_data.get("members", [])
    flattened_members = [flatten_member(member) for member in members_list]
    return pd.DataFrame(flattened_members)


def save_data_as_csv_and_json(df: pd.DataFrame, csv_filename: str, json_filename: str) -> None:

    df.to_csv(csv_filename, sep=",", index=False, encoding="utf-8-sig")
    df.to_json(json_filename, orient="records", force_ascii=False, indent=4)
    print(f"Данные успешно сохранены в файлы: {csv_filename} и {json_filename}")



groups = [
        "public101972636",
        "club106674208",
        "club119689350",
        "club109999891",
        "public147087864",
        "club78015768"
    ]


access_token = r"fa2c7749fa2c7749fa2c77496bf9014108ffa2cfa2c77499df7719b52bddb7a28973191"


print("Начинаем парсинг участников...")
parsing_result = fetch_groups_members(groups, access_token)


print("Преобразуем данные (flatten) и создаём DataFrame...")
df = flatten_members_data(parsing_result)


csv_filename = "members_flat.csv"
json_filename = "members_flat.json"
print(f"Сохраняем результат в CSV ({csv_filename}) и JSON ({json_filename})...")
save_data_as_csv_and_json(df, csv_filename, json_filename)

print("Парсинг и преобразование данных завершены!")

Начинаем парсинг участников...
Преобразуем данные (flatten) и создаём DataFrame...
Сохраняем результат в CSV (members_flat.csv) и JSON (members_flat.json)...
Данные успешно сохранены в файлы: members_flat.csv и members_flat.json
Парсинг и преобразование данных завершены!


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


df = pd.read_csv("members_flat.csv", sep=',', encoding='utf-8-sig')


def parse_age(bdate: str):
    if pd.isna(bdate):
        return None
    parts = bdate.split('.')

    if len(parts) == 3 and parts[2].isdigit():
        year = int(parts[2])
    elif len(parts) == 1 and parts[0].isdigit() and len(parts[0]) == 4:

        year = int(parts[0])
    else:
        return None

    current_year = datetime.now().year
    age = current_year - year
    return age if age > 0 else None

df['age'] = df['bdate'].apply(parse_age)


plt.figure()
sex_filtered = df.loc[df['sex'] != 'не указан', 'sex']
sex_filtered.value_counts(dropna=False).plot(kind='bar')
plt.title("Распределение пользователей по полу")
plt.xlabel("Пол")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_sex_distribution.png")
plt.close()


ages = df['age'].dropna()
plt.figure()
plt.hist(ages, bins=range(int(ages.min()), int(ages.max())+2, 2))
plt.title("Распределение по возрасту")
plt.xlabel("Возраст")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_age_distribution.png")
plt.close()


plt.figure()
relation_filtered = df.loc[df['relation'] != 'не указано', 'relation']
relation_filtered.value_counts(dropna=False).plot(kind='bar')
plt.title("Распределение по семейному положению")
plt.xlabel("Семейное положение")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_relation_distribution.png")
plt.close()


plt.figure()
political_filtered = df.loc[df['political'] != 'не указано', 'political']
political_filtered.value_counts(dropna=False).plot(kind='bar')
plt.title("Политические взгляды")
plt.xlabel("Политические взгляды")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_political_distribution.png")
plt.close()


plt.figure()
life_main_filtered = df.loc[df['life_main'] != 'не указано', 'life_main']
life_main_filtered.value_counts(dropna=False).plot(kind='bar')
plt.title("Главные ценности в жизни")
plt.xlabel("Главные ценности")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_life_main_distribution.png")
plt.close()


plt.figure()
smoking_filtered = df.loc[df['smoking'] != 'не указано', 'smoking']
smoking_filtered.value_counts(dropna=False).plot(kind='bar')
plt.title("Отношение к курению")
plt.xlabel("Отношение к курению")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_smoking_distribution.png")
plt.close()


plt.figure()
alcohol_filtered = df.loc[df['alcohol'] != 'не указано', 'alcohol']
alcohol_filtered.value_counts(dropna=False).plot(kind='bar')
plt.title("Отношение к алкоголю")
plt.xlabel("Отношение к алкоголю")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_alcohol_distribution.png")
plt.close()


plt.figure()
uni_filtered = df.loc[df['university_name'] != 'не указано', 'university_name']
uni_filtered.value_counts(dropna=False).head(10).plot(kind='bar')
plt.title("Топ-10 университетов")
plt.xlabel("Университет")
plt.ylabel("Количество пользователей")
plt.tight_layout()
plt.savefig("plots_top_universities.png")
plt.close()


<ipython-input-2-27267dcc377e>:112: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
